# Tools

> 
> Небольшие инструменты для разработки датасетов
> 

##### Печать структуры дерева папок

In [5]:
from pathlib import Path

def print_dirs_only(root, max_depth=5, prefix=''):
    root_path = Path(root)
    
    if not root_path.exists():
        print(f"Путь не существует: {root}")
        return
    
    for item in sorted(root_path.iterdir()):
        if item.is_dir():  # Только директории
            print(prefix + '|-- ' + item.name + '/')
            if max_depth > 0:
                print_dirs_only(item, max_depth - 1, prefix + '|   ')

print_dirs_only('datasets', max_depth=3)


|-- PipeBoxSegmentation/
|   |-- images/
|   |   |-- train/
|   |   |-- val/
|   |-- masks/
|   |   |-- train/
|   |   |-- val/
|-- PipeBoxSegmentation_augmented/
|   |-- images/
|   |   |-- train/
|   |   |-- val/
|   |-- masks/
|   |   |-- train/
|   |   |-- val/
|-- PipeSegmentation/
|   |-- images/
|   |   |-- train/
|   |   |-- val/
|   |-- masks/
|   |   |-- train/
|   |   |-- val/
|   |-- poligones/
|-- PipeSegmentation_augmented/
|   |-- images/
|   |   |-- train/
|   |   |-- val/
|   |-- masks/
|   |   |-- train/
|   |   |-- val/


##### Создание масок по разметке формата YOLO seg

In [ ]:
import os
from PIL import Image, ImageDraw

# Пути к папкам
images_folder = "datasets/PipeSegmentation/images/train"        # исходные картинки
labels_folder = "datasets/PipeSegmentation/poligones"        # YOLO seg файлы
output_folder = "datasets/PipeSegmentation/masks/train"         # куда сохранять маски

os.makedirs(output_folder, exist_ok=True)

for img_file in os.listdir(images_folder):
    if not img_file.lower().endswith((".jpg", ".png")):
        continue

    img_path = os.path.join(images_folder, img_file)
    label_path = os.path.join(labels_folder, os.path.splitext(img_file)[0] + ".txt")
    mask_path = os.path.join(output_folder, os.path.splitext(img_file)[0] + "_mask.png")

    # Открываем изображение для определения размеров
    with Image.open(img_path) as img:
        width, height = img.size

    # Создаем черную маску
    mask = Image.new("L", (width, height), 0)  # L - grayscale, 0 = черный
    draw = ImageDraw.Draw(mask)

    # Проверяем наличие файла разметки
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            for line in f:
                parts = list(map(float, line.strip().split()))
                if len(parts) < 3:
                    continue  # некорректная строка

                # Пропускаем класс
                coords = parts[1:]  # x1 y1 x2 y2 ...
                # Преобразуем координаты из нормализованных [0-1] в пиксели
                poly = [(coords[i] * width, coords[i+1] * height) for i in range(0, len(coords), 2)]
                draw.polygon(poly, fill=255)  # 255 = белый

    mask.save(mask_path)
    print(f"✅ Маска создана: {mask_path}")

print("Все маски сгенерированы!")

##### Копирование файлов из папки 1 в папку 2, если названия есть в папке 3

In [ ]:
import os
import shutil
from tqdm import tqdm

# Пути к папкам
folder1 = "datasets/PipeBoxSegmentation/images/train"   # откуда копируем
folder2 = "datasets/PipeSegmentation/poligones"   # с какими сравниваем имена
folder3 = "datasets/PipeSegmentation/images/train"   # куда копируем

os.makedirs(folder3, exist_ok=True)

# Получаем множество имён без расширений из второй папки
names_in_folder2 = {
    os.path.splitext(f)[0] for f in os.listdir(folder2)
    if os.path.isfile(os.path.join(folder2, f))
}

# Проходим по файлам первой папки и копируем совпадающие
for filename in tqdm(os.listdir(folder1), desc="Копирование файлов"):
    name, ext = os.path.splitext(filename)
    src_path = os.path.join(folder1, filename)

    if os.path.isfile(src_path) and name in names_in_folder2:
        dst_path = os.path.join(folder3, filename)
        shutil.copy2(src_path, dst_path)  # copy2 сохраняет метаданные


Копирование файлов:   0%|          | 0/1103 [00:00<?, ?it/s]

Копирование файлов: 100%|██████████| 1103/1103 [00:00<00:00, 13242.68it/s]


##### Удаление из названий файлов в директории суффиксов

Превращение файлов с названием `<6 цифр>_<что-то>.<расширение>` в `<6 цифр>.<расширение>`

In [ ]:
import os
import re

def rename_files_in_directory(directory_path):
    """
    Переименовывает файлы в директории по шаблону:
    Исходное имя: <6 цифр>_<что-то>.<расширение>
    Новое имя: <6 цифр>.<расширение>
    
    Args:
        directory_path (str): Путь к директории с файлами
    """
    # Проверяем существование директории
    if not os.path.exists(directory_path):
        raise FileNotFoundError(f"Директория не найдена: {directory_path}")
    
    
    if not os.path.isdir(directory_path):
        raise ValueError(f"Указанный путь не является директорией: {directory_path}")
    
    # Регулярное выражение для поиска шаблона
    pattern = re.compile(r'^(\d{6})_.*(\.[^.]+)$')
    
    
    for filename in os.listdir(directory_path):
        # Проверяем, что это файл (не директория)
        filepath = os.path.join(directory_path, filename)
        if not os.path.isfile(filepath):
            continue
            
        # Ищем соответствие шаблону
        match = pattern.match(filename)
        if match:
            # Получаем 6 цифр и расширение
            digits = match.group(1)
            extension = match.group(2)
            
            # Формируем новое имя
            new_filename = f"{digits}{extension}"
            new_filepath = os.path.join(directory_path, new_filename)
            
            # Переименовываем файл
            try:
                os.rename(filepath, new_filepath)
                print(f"Переименовано: {filename} → {new_filename}")
            except Exception as e:
                print(f"Ошибка при переименовании {filename}: {e}")


rename_files_in_directory("datasets/PipeSegmentation/images/train")

##### Деление датасета на тренировочную и валидационную части

Перевод 20% датасета в валидационную часть 

In [15]:
import os
import shutil
import random
from tqdm import tqdm

# Путь к датасету
dataset_path = "datasets/PipeSegmentation"

# Папки с изображениями и масками
images_train = os.path.join(dataset_path, "images/train")
images_val = os.path.join(dataset_path, "images/val")
masks_train = os.path.join(dataset_path, "masks/train")
masks_val = os.path.join(dataset_path, "masks/val")

# Создаём папки val, если их нет
os.makedirs(images_val, exist_ok=True)
os.makedirs(masks_val, exist_ok=True)

# Получаем список файлов изображений
image_files = [f for f in os.listdir(images_train) if f.lower().endswith(".jpg")]

# Выбираем 20% случайных файлов
num_to_move = int(len(image_files) * 0.2)
files_to_move = random.sample(image_files, num_to_move)

for img_file in tqdm(files_to_move, desc="Перемещение файлов"):
    # Перемещаем изображение
    shutil.move(os.path.join(images_train, img_file),
                os.path.join(images_val, img_file))
    
    # Перемещаем соответствующую маску (.png, _mask)
    name, _ = os.path.splitext(img_file)
    mask_file = f"{name}_mask.png"
    
    if os.path.exists(os.path.join(masks_train, mask_file)):
        shutil.move(os.path.join(masks_train, mask_file),
                    os.path.join(masks_val, mask_file))
    else:
        print(f"⚠️ Маска не найдена: {mask_file}")

print(f"✅ Перемещено {num_to_move} файлов ({len(image_files)} всего)")


Перемещение файлов: 100%|██████████| 17/17 [00:00<00:00, 364.15it/s]

✅ Перемещено 17 файлов (85 всего)


##### Аугментирование данных

Ячейка для аугментирования данных для семантической сегментации

In [ ]:
import os
import cv2
import random
import numpy as np
import albumentations as A
from tqdm import tqdm
import torch

# ==========================================================
# Фиксируем все random seed'ы
# ==========================================================
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# Для повторяемости при работе DataLoader/torch
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ==========================================================
# --- Настройки путей ---
# ==========================================================
DATASET_NAME = "PipeSegmentation"
SPLITS = ["train", "val"]

BASE_DIR = f"datasets/{DATASET_NAME}"
BASE_OUTPUT_DIR = f"datasets/{DATASET_NAME}_augmented"

IMAGE_DIRS = {split: os.path.join(BASE_DIR, "images", split) for split in SPLITS}
MASK_DIRS = {split: os.path.join(BASE_DIR, "masks", split) for split in SPLITS}
OUTPUT_IMAGE_DIRS = {split: os.path.join(BASE_OUTPUT_DIR, "images", split) for split in SPLITS}
OUTPUT_MASK_DIRS = {split: os.path.join(BASE_OUTPUT_DIR, "masks", split) for split in SPLITS}

# Создаём выходные директории
for split in SPLITS:
    os.makedirs(OUTPUT_IMAGE_DIRS[split], exist_ok=True)
    os.makedirs(OUTPUT_MASK_DIRS[split], exist_ok=True)

# ==========================================================
# --- Аугментации (детерминированные) ---
# ==========================================================
transform = A.Compose(
    [
        A.OneOf([
            A.GaussNoise(var_limit=(10.0, 10.0), p=1.0),
            A.ISONoise(p=1.0)
        ], p=0.5),
        A.Rotate(limit=10, p=0.7),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.8)
    ],
    additional_targets={"mask": "mask"}
)

# ==========================================================
# --- Функция аугментации и сохранения ---
# ==========================================================
def augment_and_save(image_path, mask_path, out_image_dir, out_mask_dir, filename, n_aug=3):
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    if image is None or mask is None:
        print(f"⚠️ Ошибка чтения: {image_path} / {mask_path}")
        return

    if image.shape[:2] != mask.shape[:2]:
        print(f"⚠️ Несовпадение размеров: {image_path} и {mask_path}")
        return

    base_name = os.path.splitext(filename)[0]

    for i in range(n_aug):
        # Используем собственный seed для каждого augment'а, чтобы порядок не влиял
        local_seed = SEED + i
        random.seed(local_seed)
        np.random.seed(local_seed)

        augmented = transform(image=image, mask=mask)
        aug_image = augmented["image"]
        aug_mask = augmented["mask"]

        cv2.imwrite(os.path.join(out_image_dir, f"{base_name}_aug{i}.jpg"), aug_image)
        cv2.imwrite(os.path.join(out_mask_dir, f"{base_name}_aug{i}.png"), aug_mask)

# ==========================================================
# --- Основной цикл ---
# ==========================================================
for split in SPLITS:
    image_files = [f for f in os.listdir(IMAGE_DIRS[split]) if f.lower().endswith(".jpg")]

    for img_file in tqdm(image_files, desc=f"Аугментация {split}"):
        image_path = os.path.join(IMAGE_DIRS[split], img_file)
        mask_file = os.path.splitext(img_file)[0] + "_mask.png"
        mask_path = os.path.join(MASK_DIRS[split], mask_file)

        if not os.path.exists(mask_path):
            print(f"⚠️ Маска не найдена: {mask_path}")
            continue

        augment_and_save(
            image_path,
            mask_path,
            OUTPUT_IMAGE_DIRS[split],
            OUTPUT_MASK_DIRS[split],
            img_file,
            n_aug=3
        )


Аугментация val: 100%|██████████| 17/17 [00:01<00:00, 11.58it/s]

✅ Аугментация завершена!
